In [1]:
import pandas as pd
import shutil
import os
import sys
import cv2
import numpy as np

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.head

<bound method NDFrame.head of       Unnamed: 0      name   x1   x2    y1   y2                 classname  \
0          14938  4173.png  265  302   292  355            face_with_mask   
1          12845  2798.png  773    5  1020  586            face_with_mask   
2           6436  2097.jpg  601  172   727  349       face_other_covering   
3              5  6427.png  218   98   577  580  face_with_mask_incorrect   
4          10867  4511.png  297    6   452  240            face_with_mask   
...          ...       ...  ...  ...   ...  ...                       ...   
5084       12279  3945.png  214  150   316  237            face_with_mask   
5085       15197  5346.jpg  253  155   493  389       face_other_covering   
5086       14559  4193.png  220   60   416  333            face_with_mask   
5087       12899  3525.png   48    9   379  371              face_no_mask   
5088       12052  6036.jpg  650  108   755  233       face_other_covering   

      face_no_mask  face_other_covering  face

In [4]:
correct_mask = df[df["classname"] == "face_with_mask"]

In [5]:
correct_mask = correct_mask.drop_duplicates(subset=["name"])

In [6]:
correct_mask.drop(['x1', 'x2', 'y1', 'y2'], axis='columns', inplace=True)

In [7]:
correct_mask.head

<bound method NDFrame.head of       Unnamed: 0      name       classname  face_no_mask  face_other_covering  \
0          14938  4173.png  face_with_mask             0                    0   
1          12845  2798.png  face_with_mask             0                    0   
4          10867  4511.png  face_with_mask             0                    0   
6          14990  5108.jpg  face_with_mask             0                    0   
8           6961  2168.png  face_with_mask             0                    0   
...          ...       ...             ...           ...                  ...   
5057        1116  3540.png  face_with_mask             0                    0   
5064        4373  3129.png  face_with_mask             0                    0   
5071        5930  2187.png  face_with_mask             0                    0   
5080        4396  5014.jpg  face_with_mask             0                    0   
5086       14559  4193.png  face_with_mask             0                    0  

In [8]:
nomask = df[df["classname"] == "face_no_mask"]

In [9]:
nomask = nomask.drop_duplicates(subset=["name"])

In [10]:
nomask.drop(['x1', 'x2', 'y1', 'y2'], axis='columns', inplace=True)

In [11]:
incorrect_mask = df[df["classname"] == "face_with_mask_incorrect"]

In [12]:
incorrect_mask = incorrect_mask.drop_duplicates(subset=["name"])

In [13]:
incorrect_mask.drop(['x1', 'x2', 'y1', 'y2'], axis='columns', inplace=True)

In [14]:
incorrect_mask.head()

,Unnamed: 0,name,classname,face_no_mask,face_other_covering,face_with_mask,face_with_mask_incorrect
3,5,6427.png,face_with_mask_incorrect,0,0,0,1
7,11133,3063.png,face_with_mask_incorrect,0,0,0,1
187,11189,2812.png,face_with_mask_incorrect,0,0,0,1
243,8186,2626.png,face_with_mask_incorrect,0,0,0,1
361,3061,6290.jpg,face_with_mask_incorrect,0,0,0,1


In [15]:
#adding images to incorrect_mask category
from os import walk
root_dir = os.path.abspath('.')
SuppImagesDir = os.path.join(root_dir, 'supplementary_images/Improperly Worn')
ImagesDir = os.path.join(root_dir, 'croppedv2')

f=[]
if not os.path.exists(ImagesDir):
            os.mkdir(ImagesDir)
for (dirpath, dirnames, filenames) in walk(SuppImagesDir):
        f.extend(filenames)
        break
#will be appended to the dataframe after cropping
for filenames in f:
    src_path = SuppImagesDir + "/" + filenames
    dst_path = ImagesDir + "/" + filenames
    shutil.copy(src_path, dst_path)

In [16]:
df.classname.unique()

array(['face_with_mask', 'face_other_covering',
       'face_with_mask_incorrect', 'face_no_mask'], dtype=object)

In [17]:
accepted_classes = ['face_with_mask', 'face_no_mask',
       'face_with_mask_incorrect', 'face_other_covering']

classes_df = df[df["classname"].isin(accepted_classes)]

In [18]:
final_class = classes_df
final_class

,Unnamed: 0,name,x1,x2,y1,y2,classname,face_no_mask,face_other_covering,face_with_mask,face_with_mask_incorrect
0,14938,4173.png,265,302,292,355,face_with_mask,0,0,1,0
1,12845,2798.png,773,5,1020,586,face_with_mask,0,0,1,0
2,6436,2097.jpg,601,172,727,349,face_other_covering,0,1,0,1
3,5,6427.png,218,98,577,580,face_with_mask_incorrect,0,0,0,1
4,10867,4511.png,297,6,452,240,face_with_mask,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
5084,12279,3945.png,214,150,316,237,face_with_mask,0,0,1,0
5085,15197,5346.jpg,253,155,493,389,face_other_covering,0,1,0,1
5086,14559,4193.png,220,60,416,333,face_with_mask,0,0,1,0
5087,12899,3525.png,48,9,379,371,face_no_mask,1,0,0,0


In [19]:
class_bool = pd.get_dummies(classes_df["classname"])

In [20]:
final_class = classes_df.join(class_bool[["face_no_mask", "face_other_covering", "face_with_mask", "face_with_mask_incorrect"]])

ValueError: columns overlap but no suffix specified: Index(['face_no_mask', 'face_other_covering', 'face_with_mask',
       'face_with_mask_incorrect'],
      dtype='object')

In [ ]:
final_class

In [21]:
final_class[final_class["face_other_covering"] == 1]

,Unnamed: 0,name,x1,x2,y1,y2,classname,face_no_mask,face_other_covering,face_with_mask,face_with_mask_incorrect
2,6436,2097.jpg,601,172,727,349,face_other_covering,0,1,0,1
5,12391,4095.png,319,143,524,436,face_other_covering,0,1,0,1
9,5657,3523.png,70,0,378,444,face_other_covering,0,1,0,1
12,10966,2993.png,79,88,259,296,face_other_covering,0,1,0,1
13,12298,5270.jpg,206,31,270,108,face_other_covering,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
5081,7623,5730.jpg,168,40,217,102,face_other_covering,0,1,0,1
5082,8130,1861.jpg,518,116,938,632,face_other_covering,0,1,0,1
5083,7329,2119.jpg,386,88,803,547,face_other_covering,0,1,0,1
5085,15197,5346.jpg,253,155,493,389,face_other_covering,0,1,0,1


In [22]:
final_class.loc[(final_class["face_with_mask_incorrect"] == 1) | (final_class["face_other_covering"] == 1), "face_with_mask_incorrect"] = 1

In [23]:
final_class[(final_class["face_other_covering"] == 1) & (final_class["face_with_mask_incorrect"] == 0)]

,Unnamed: 0,name,x1,x2,y1,y2,classname,face_no_mask,face_other_covering,face_with_mask,face_with_mask_incorrect


In [24]:
final_class.shape

(5089, 11)

In [25]:
final_class.head()

,Unnamed: 0,name,x1,x2,y1,y2,classname,face_no_mask,face_other_covering,face_with_mask,face_with_mask_incorrect
0,14938,4173.png,265,302,292,355,face_with_mask,0,0,1,0
1,12845,2798.png,773,5,1020,586,face_with_mask,0,0,1,0
2,6436,2097.jpg,601,172,727,349,face_other_covering,0,1,0,1
3,5,6427.png,218,98,577,580,face_with_mask_incorrect,0,0,0,1
4,10867,4511.png,297,6,452,240,face_with_mask,0,0,1,0


In [26]:
cropping = final_class.copy()
cropping.drop(['face_no_mask', 'face_other_covering', 'face_with_mask', 'face_with_mask_incorrect'], axis='columns', inplace=True)


In [27]:
cropping = cropping[(cropping['x1']!=cropping['y1']) & (cropping['x2']!=cropping['y2'])]

In [28]:
cropping["index"] = range(0, len(cropping))
cropping["newFilename"] = np.nan
cropping.reset_index()
cropping = cropping.set_index("index")

In [29]:
cropping.head()

,Unnamed: 0,name,x1,x2,y1,y2,classname,newFilename
index,,,,,,,,
0,14938,4173.png,265,302,292,355,face_with_mask,NaN
1,12845,2798.png,773,5,1020,586,face_with_mask,NaN
2,6436,2097.jpg,601,172,727,349,face_other_covering,NaN
3,5,6427.png,218,98,577,580,face_with_mask_incorrect,NaN
4,10867,4511.png,297,6,452,240,face_with_mask,NaN


In [30]:
cropping = cropping.fillna(0)
cropping.x1.astype(int)
cropping.x2.astype(int)
cropping.y1.astype(int)
cropping.y2.astype(int)


index
0       355
1       586
2       349
3       580
4       240
       ... 
5084    237
5085    389
5086    333
5087    371
5088    233
Name: y2, Length: 5089, dtype: int32

In [31]:
cropping.head()

,Unnamed: 0,name,x1,x2,y1,y2,classname,newFilename
index,,,,,,,,
0,14938,4173.png,265,302,292,355,face_with_mask,0.0
1,12845,2798.png,773,5,1020,586,face_with_mask,0.0
2,6436,2097.jpg,601,172,727,349,face_other_covering,0.0
3,5,6427.png,218,98,577,580,face_with_mask_incorrect,0.0
4,10867,4511.png,297,6,452,240,face_with_mask,0.0


In [32]:
#Cropping images according to ROI 

root_dir = os.path.abspath('.')
ImagesDir = os.path.join(root_dir, "images//")
croppedDir= os.path.join(root_dir, "croppedv2")

index_counter = 0

print(ImagesDir)

print("Cropping images....")
for index, row in cropping.iterrows(): 
    if not os.path.exists(croppedDir):
            os.mkdir(croppedDir)
    img = cv2.imread(ImagesDir + row["name"])
    if img is None:
        cropping.drop(index)
        print(row["name"])
        continue
    try:
        img_cropped = img[row["x2"]:row["y2"], row["x1"]:row["y1"]]
        new_filename = str(row["x1"]) + row["name"]
        cv2.imwrite(croppedDir +  '/' + new_filename, img_cropped)
        cropping.loc[index, "newFilename"] = new_filename

    except Exception as e:
        print(e)
        
    index_counter += 1
    
print("Images cropped successfully...")

C:\Users\dcsam\Documents\programming\thesis\DatasetCleaning\images//
Cropping images....
4173.png
2798.png
2097.jpg
6427.png
4511.png
4095.png
5108.jpg
3063.png
2168.png
3523.png
2154.png
5989.jpg
2993.png
5270.jpg
3898.png
4228.png
2544.png
6392.jpg
2054.jpg
2151.jpg
5364.jpg
6378.jpg
3138.png
6154.jpg
4588.png
4141.png
3082.png
6385.jpg
3305.png
3334.png
2063.jpg
6314.jpg
2510.png
5483.jpg
3856.png
5983.jpg
6255.jpg
4866.png
5507.jpg
4299.png
4645.png
4491.png
3825.png
5972.jpg
6316.jpg
5298.jpg
5752.jpg
4817.png
3996.png
4450.png
5899.jpg
4799.png
6270.jpg
3820.png
5172.jpg
1982.png
3587.png
5448.jpg
4381.png
4676.png
3638.png
3582.png
4064.png
5206.jpg
2239.png
4439.png
6232.jpg
1891.jpg
2439.png
3822.png
2889.png
4733.png
3690.png
5619.jpg
5456.jpg
2409.png
5990.jpg
5946.jpg
2617.png
4971.png
2909.png
2198.png
5409.jpg
6288.jpg
3293.png
2016.jpg
4751.png
3144.png
4356.png
4098.png
2205.png
4655.png
5033.jpg
6293.jpg
3947.png
3536.png
3838.png
6007.jpg
6292.jpg
3191.png
3468.png
28

3793.png
4925.png
3067.png
5062.jpg
2492.png
4688.png
3436.png
2675.png
2319.png
4655.png
6392.jpg
4186.png
2953.png
2986.png
1879.jpg
5713.jpg
6000.jpg
5286.jpg
2688.png
3843.png
6271.jpg
3994.png
2540.png
4613.png
3793.png
5059.jpg
6041.jpg
4811.png
5247.jpg
5910.jpg
6102.jpg
3166.png
4980.png
4908.png
2258.png
5385.jpg
2324.png
3774.png
4188.png
4513.png
3757.png
5047.jpg
2092.jpg
5916.jpg
6212.jpg
4010.png
5362.jpg
3981.png
3761.png
6247.jpg
6113.jpg
5619.jpg
3961.png
6103.jpg
4306.png
5095.jpg
5376.jpg
4461.png
4115.png
1972.jpg
5375.jpg
5039.jpg
1835.jpg
2277.png
6259.jpeg
5246.jpg
3993.png
5108.jpg
4506.png
5042.jpg
5219.jpg
3474.png
2020.jpg
2181.png
1938.jpg
3640.png
3013.png
6007.jpg
2213.png
4645.png
6033.jpg
5317.jpg
5077.jpg
3223.png
2507.png
3294.png
3814.png
5949.jpg
2935.png
2386.png
2965.png
2143.jpg
5431.jpg
3222.png
4565.png
4625.png
3578.png
2878.png
2753.png
3775.png
5179.jpg
4174.png
4775.png
3698.png
6217.jpg
1914.jpg
4099.png
5949.jpg
6006.jpg
6166.png
4728.png


2546.png
6013.jpg
5671.jpg
4181.png
3393.png
3955.png
5947.jpg
6162.jpg
5030.jpg
6194.jpg
1914.jpg
5106.jpg
4517.png
4866.png
4175.png
3234.png
2016.jpg
4845.png
4369.png
2275.png
6082.jpg
5425.jpg
2525.png
3181.png
3301.png
4909.png
4386.png
4088.png
1978.jpg
5088.jpg
6211.jpg
6005.jpg
2422.png
2846.png
2521.png
2833.png
5611.jpg
5499.jpg
3669.png
2813.png
6054.png
2670.png
4141.png
5009.jpg
2094.jpg
3607.png
5965.jpg
2134.jpg
3978.png
2419.png
2739.png
3275.png
5466.jpg
2098.jpg
2935.png
6370.jpg
6043.jpg
3356.png
3599.png
4569.png
5597.jpg
4455.png
1847.jpg
4530.png
2138.jpg
3469.png
4814.png
6079.jpg
5263.jpg
6408.png
4444.png
2295.png
4590.png
3556.png
6362.jpg
2219.png
5594.jpg
2104.jpg
3027.png
4003.png
6198.jpg
5264.jpg
2092.jpg
6241.jpg
2926.png
5818.jpg
2666.png
4640.png
5453.jpg
2133.jpg
4411.png
4943.png
5871.jpg
6142.jpg
5728.jpg
3445.png
5897.jpg
5090.jpg
5537.jpg
2422.png
4119.png
2038.jpg
4162.png
4047.png
3361.png
5999.jpg
4400.png
2679.png
5595.jpg
4877.png
3313.png
6

5245.jpg
6088.jpg
2053.jpg
5740.jpg
2988.png
4653.png
3862.png
4060.png
3311.png
6042.jpg
2333.png
2935.png
2691.png
4493.png
3329.png
3177.png
5291.jpg
4717.png
3522.png
4943.png
4025.png
3271.png
5703.jpg
6251.jpg
4411.png
2834.png
5072.jpg
1853.jpg
4260.png
3660.png
6317.jpg
3776.png
4089.png
6256.jpg
4230.png
4697.png
3733.png
4814.png
3643.png
4083.png
5361.jpg
3140.png
4550.png
6337.jpg
4378.png
4504.png
3748.png
3315.png
5923.jpg
3679.png
2846.png
1849.jpg
2435.png
6228.jpg
3786.png
6197.jpg
2877.png
4604.png
1892.jpg
1870.jpg
4223.png
5635.jpg
3030.png
3420.png
3424.png
5618.jpg
4604.png
4210.png
2397.png
5300.jpeg
3358.png
6006.jpg
6102.jpg
5612.jpg
3369.png
3645.png
3581.png
5219.jpg
5669.jpg
2674.png
4141.png
1951.jpg
4943.png
4660.png
2443.png
3934.png
5424.jpg
1888.jpg
2977.png
6248.jpg
6285.jpg
4171.png
2103.jpg
2792.png
3709.png
4579.png
3689.png
5217.jpg
4071.png
3698.png
5058.jpg
4377.png
3162.png
6065.jpg
3971.png
4862.png
3468.png
2277.png
3224.png
4418.png
5135.png


KeyboardInterrupt: 

In [230]:
cropping[cropping['newFilename']== '6485107.jpg']

,Unnamed: 0,name,x1,x2,y1,y2,classname,newFilename
index,,,,,,,,


In [231]:
print(f)

['00000_Mask_Mouth_Chin.jpg', '00001_Mask_Chin.jpg', '00001_Mask_Mouth_Chin.jpg', '00002_Mask_Mouth_Chin.jpg', '00003_Mask_Mouth_Chin.jpg', '00004_Mask_Mouth_Chin.jpg', '00005_Mask_Mouth_Chin.jpg', '00007_Mask_Mouth_Chin.jpg', '00008_Mask_Mouth_Chin.jpg', '00010_Mask_Mouth_Chin.jpg', '00011_Mask_Mouth_Chin.jpg', '00012_Mask_Chin.jpg', '00013_Mask_Mouth_Chin.jpg', '00014_Mask_Mouth_Chin.jpg', '00015_Mask_Mouth_Chin.jpg', '00016_Mask_Mouth_Chin.jpg', '00017_Mask_Chin.jpg', '00018_Mask_Mouth_Chin.jpg', '00019_Mask_Chin.jpg', '00020_Mask_Mouth_Chin.jpg', '00021_Mask_Mouth_Chin.jpg', '00022_Mask_Mouth_Chin.jpg', '00023_Mask_Mouth_Chin.jpg', '00025_Mask_Mouth_Chin.jpg', '00026_Mask_Mouth_Chin.jpg', '00027_Mask_Chin.jpg', '00029_Mask_Chin.jpg', '00030_Mask_Mouth_Chin.jpg', '00031_Mask_Mouth_Chin.jpg', '00032_Mask_Mouth_Chin.jpg', '00033_Mask_Mouth_Chin.jpg', '00034_Mask_Mouth_Chin.jpg', '00036_Mask_Nose_Mouth.jpg', '00037_Mask_Chin.jpg', '00038_Mask_Mouth_Chin.jpg', '00039_Mask_Mouth_Chin.jpg

In [232]:
#append the supplementary images to the dataframe (TBC)
to_append = f
supp_images = pd.DataFrame(f, columns = ['name'])
supp_images['x1'] = 0
supp_images['x2'] = 128
supp_images['y1'] = 0
supp_images['y2'] = 128
supp_images['classname'] = 'face_with_mask_incorrect'
supp_images['newFilename'] = pd.DataFrame(f, columns = ['name'])

In [233]:
supp_images.head()

,name,x1,x2,y1,y2,classname,newFilename
0,00000_Mask_Mouth_Chin.jpg,0,128,0,128,face_with_mask_incorrect,00000_Mask_Mouth_Chin.jpg
1,00001_Mask_Chin.jpg,0,128,0,128,face_with_mask_incorrect,00001_Mask_Chin.jpg
2,00001_Mask_Mouth_Chin.jpg,0,128,0,128,face_with_mask_incorrect,00001_Mask_Mouth_Chin.jpg
3,00002_Mask_Mouth_Chin.jpg,0,128,0,128,face_with_mask_incorrect,00002_Mask_Mouth_Chin.jpg
4,00003_Mask_Mouth_Chin.jpg,0,128,0,128,face_with_mask_incorrect,00003_Mask_Mouth_Chin.jpg


In [234]:
cropping=cropping.drop("Unnamed: 0", axis=1)


In [235]:
cropping = pd.concat([cropping, supp_images])

In [236]:
cropping.head()

,name,x1,x2,y1,y2,classname,newFilename
0,4173.png,265,302,292,355,face_with_mask,2654173.png
1,2798.png,773,5,1020,586,face_with_mask,7732798.png
2,2097.jpg,601,172,727,349,face_other_covering,6012097.jpg
3,6427.png,218,98,577,580,face_with_mask_incorrect,2186427.png
4,4511.png,297,6,452,240,face_with_mask,2974511.png


In [238]:
#image resizing 
croppedDir= os.path.join(root_dir, "croppedv2")
print(croppedDir)
#variables for image width and height
width = 280
height = 280
dim = (width, height)

print("Resizing images....")
for name, x1, x2, y1, y2, classname, newFilename in cropping.values: 
    img = cv2.imread(croppedDir + '\\' + newFilename)
    try:
        img_resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        cv2.imwrite(croppedDir + '/' + newFilename, img_resized)
        print(newFilename)
        print(img.shape())
    except Exception as e:
        print(newFilename)
        print(x2, y2, x1, y1)
        print(e)
print("Images resized successfully...")      

C:\Users\rogin\Desktop\Capsnet\capsnetMask\croppedv2
Resizing images....
2654173.png
2654173.png
302 355 265 292
'tuple' object is not callable
7732798.png
7732798.png
5 586 773 1020
'tuple' object is not callable
6012097.jpg
6012097.jpg
172 349 601 727
'tuple' object is not callable
2186427.png
2186427.png
98 580 218 577
'tuple' object is not callable
2974511.png
2974511.png
6 240 297 452
'tuple' object is not callable
3194095.png
3194095.png
143 436 319 524
'tuple' object is not callable
3745108.jpg
3745108.jpg
68 128 374 422
'tuple' object is not callable
4393063.png
4393063.png
72 257 439 609
'tuple' object is not callable
3212168.png
3212168.png
114 385 321 535
'tuple' object is not callable
703523.png
703523.png
0 444 70 378
'tuple' object is not callable
1112154.png
1112154.png
281 966 111 586
'tuple' object is not callable
8845989.jpg
8845989.jpg
174 302 884 986
'tuple' object is not callable
792993.png
792993.png
88 296 79 259
'tuple' object is not callable
2065270.jpg
2065270

2773584.png
2773584.png
428 467 277 307
'tuple' object is not callable
7255225.jpg
7255225.jpg
86 732 725 1224
'tuple' object is not callable
426392.jpg
426392.jpg
232 302 42 94
'tuple' object is not callable
05963.jpg
05963.jpg
365 502 0 97
'tuple' object is not callable
3076013.jpg
3076013.jpg
115 197 307 364
'tuple' object is not callable
4512739.png
4512739.png
79 209 451 532
'tuple' object is not callable
2374531.png
2374531.png
184 592 237 485
'tuple' object is not callable
322685.png
322685.png
251 371 32 120
'tuple' object is not callable
8545557.jpg
8545557.jpg
51 1476 854 1905
'tuple' object is not callable
1652646.png
1652646.png
2 515 165 566
'tuple' object is not callable
675128.jpg
675128.jpg
69 134 67 111
'tuple' object is not callable
6224008.png
6224008.png
3 305 622 852
'tuple' object is not callable
10035611.jpg
10035611.jpg
123 285 1003 1118
'tuple' object is not callable
2675472.jpg
2675472.jpg
34 250 267 448
'tuple' object is not callable
2671850.jpg
2671850.jpg
2

283934.png
283934.png
50 112 28 75
'tuple' object is not callable
74672.png
74672.png
372 637 7 222
'tuple' object is not callable
7325452.jpg
7325452.jpg
90 842 732 1269
'tuple' object is not callable
4714778.png
4714778.png
174 239 471 523
'tuple' object is not callable
2535971.jpg
2535971.jpg
46 150 253 358
'tuple' object is not callable
3043159.png
3043159.png
7 338 304 541
'tuple' object is not callable
3234086.png
3234086.png
169 208 323 356
'tuple' object is not callable
8425175.jpg
8425175.jpg
263 533 842 1038
'tuple' object is not callable
574660.png
574660.png
32 98 57 114
'tuple' object is not callable
3425171.jpg
3425171.jpg
33 111 342 400
'tuple' object is not callable
973205.png
973205.png
146 671 97 460
'tuple' object is not callable
62304.png
62304.png
4 442 6 674
'tuple' object is not callable
23334.png
23334.png
0 110 2 72
'tuple' object is not callable
4212505.png
4212505.png
199 531 421 654
'tuple' object is not callable
945365.jpg
945365.jpg
0 323 94 401
'tuple' ob

2963016.png
2963016.png
282 375 296 371
'tuple' object is not callable
6053841.png
6053841.png
6 64 605 648
'tuple' object is not callable
3334219.png
3334219.png
80 162 333 388
'tuple' object is not callable
1466060.jpg
1466060.jpg
253 409 146 291
'tuple' object is not callable
2162989.png
2162989.png
2 499 216 529
'tuple' object is not callable
3523939.png
3523939.png
101 173 352 409
'tuple' object is not callable
3492492.png
3492492.png
80 126 349 387
'tuple' object is not callable
2972422.png
2972422.png
99 148 297 338
'tuple' object is not callable
4765138.jpg
4765138.jpg
156 632 476 874
'tuple' object is not callable
3362636.png
3362636.png
34 340 336 568
'tuple' object is not callable
9906211.jpg
9906211.jpg
92 232 990 1092
'tuple' object is not callable
4902141.jpg
4902141.jpg
79 262 490 653
'tuple' object is not callable
112935.png
112935.png
125 227 11 144
'tuple' object is not callable
342428.png
342428.png
119 476 34 322
'tuple' object is not callable
2762563.png
2762563.pn

1291914.jpg
1291914.jpg
312 422 129 235
'tuple' object is not callable
703074.png
703074.png
230 301 70 136
'tuple' object is not callable
7483003.png
7483003.png
9 450 748 1105
'tuple' object is not callable
2454499.png
2454499.png
90 130 245 283
'tuple' object is not callable
603946.png
603946.png
2 523 60 483
'tuple' object is not callable
245104.jpg
245104.jpg
402 545 24 129
'tuple' object is not callable
19182016.jpg
19182016.jpg
321 594 1918 2109
'tuple' object is not callable
3256187.jpg
3256187.jpg
17 152 325 457
'tuple' object is not callable
96128.jpg
96128.jpg
192 314 9 93
'tuple' object is not callable
2604040.png
2604040.png
23 177 260 383
'tuple' object is not callable
3236152.jpg
3236152.jpg
11 475 323 685
'tuple' object is not callable
226355.jpg
226355.jpg
44 213 22 175
'tuple' object is not callable
2694552.png
2694552.png
8 280 269 486
'tuple' object is not callable
714500.png
714500.png
168 323 71 225
'tuple' object is not callable
1074856.png
1074856.png
100 206 10

...

00676_Mask_Chin.jpg
00676_Mask_Chin.jpg
128 128 0 0
'tuple' object is not callable
00677_Mask_Mouth_Chin.jpg
00677_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00678_Mask_Mouth_Chin.jpg
00678_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00679_Mask_Nose_Mouth.jpg
00679_Mask_Nose_Mouth.jpg
128 128 0 0
'tuple' object is not callable
00680_Mask_Chin.jpg
00680_Mask_Chin.jpg
128 128 0 0
'tuple' object is not callable
00681_Mask_Mouth_Chin.jpg
00681_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00682_Mask_Chin.jpg
00682_Mask_Chin.jpg
128 128 0 0
'tuple' object is not callable
00683_Mask_Mouth_Chin.jpg
00683_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00684_Mask_Mouth_Chin.jpg
00684_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00685_Mask_Mouth_Chin.jpg
00685_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00686_Mask_Mouth_Chin.jpg
00686_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not 

00772_Mask_Mouth_Chin.jpg
00772_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00773_Mask_Mouth_Chin.jpg
00773_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00774_Mask_Mouth_Chin.jpg
00774_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00775_Mask_Mouth_Chin.jpg
00775_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00776_Mask_Mouth_Chin.jpg
00776_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00777_Mask_Mouth_Chin.jpg
00777_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00778_Mask_Mouth_Chin.jpg
00778_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00779_Mask_Mouth_Chin.jpg
00779_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00780_Mask_Chin.jpg
00780_Mask_Chin.jpg
128 128 0 0
'tuple' object is not callable
00781_Mask_Mouth_Chin.jpg
00781_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00782_Mask_Mouth_Chin.jpg
00782_Mask_Mouth_Chin.jpg
128 128 0 

00862_Mask_Nose_Mouth.jpg
00862_Mask_Nose_Mouth.jpg
128 128 0 0
'tuple' object is not callable
00863_Mask_Mouth_Chin.jpg
00863_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00864_Mask_Mouth_Chin.jpg
00864_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00865_Mask_Mouth_Chin.jpg
00865_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00866_Mask_Chin.jpg
00866_Mask_Chin.jpg
128 128 0 0
'tuple' object is not callable
00867_Mask_Mouth_Chin.jpg
00867_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00868_Mask_Mouth_Chin.jpg
00868_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00869_Mask_Mouth_Chin.jpg
00869_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00870_Mask_Chin.jpg
00870_Mask_Chin.jpg
128 128 0 0
'tuple' object is not callable
00871_Mask_Mouth_Chin.jpg
00871_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00872_Mask_Mouth_Chin.jpg
00872_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' ob

00958_Mask_Mouth_Chin.jpg
00958_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00959_Mask_Mouth_Chin.jpg
00959_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00961_Mask_Mouth_Chin.jpg
00961_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00962_Mask_Mouth_Chin.jpg
00962_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00963_Mask_Mouth_Chin.jpg
00963_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00964_Mask_Mouth_Chin.jpg
00964_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00965_Mask_Mouth_Chin.jpg
00965_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00966_Mask_Mouth_Chin.jpg
00966_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00967_Mask_Mouth_Chin.jpg
00967_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00968_Mask_Mouth_Chin.jpg
00968_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
00969_Mask_Mouth_Chin.jpg
00969_Mask_Mouth_Chin.jp

01058_Mask_Nose_Mouth.jpg
01058_Mask_Nose_Mouth.jpg
128 128 0 0
'tuple' object is not callable
01059_Mask_Nose_Mouth.jpg
01059_Mask_Nose_Mouth.jpg
128 128 0 0
'tuple' object is not callable
01060_Mask_Mouth_Chin.jpg
01060_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
01061_Mask_Mouth_Chin.jpg
01061_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
01062_Mask_Mouth_Chin.jpg
01062_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
01063_Mask_Nose_Mouth.jpg
01063_Mask_Nose_Mouth.jpg
128 128 0 0
'tuple' object is not callable
01064_Mask_Mouth_Chin.jpg
01064_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
01065_Mask_Mouth_Chin.jpg
01065_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
01066_Mask_Mouth_Chin.jpg
01066_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
01067_Mask_Mouth_Chin.jpg
01067_Mask_Mouth_Chin.jpg
128 128 0 0
'tuple' object is not callable
01068_Mask_Mouth_Chin.jpg
01068_Mask_Mouth_Chin.jp

In [239]:
cropping.drop(cropping[cropping.newFilename == 0.0].index, inplace=True)

In [36]:
from sklearn.model_selection import train_test_split

cropping = pd.read_csv("ds_inc.csv")

train, test = train_test_split(cropping, test_size=0.3)

In [37]:
cropping=cropping.drop("Unnamed: 0", axis=1)
cropping

,name,x1,x2,y1,y2,classname,newFilename
0,4173.png,265,302,292,355,face_with_mask,2654173.png
1,2798.png,773,5,1020,586,face_with_mask,7732798.png
2,2097.jpg,601,172,727,349,face_other_covering,6012097.jpg
3,6427.png,218,98,577,580,face_with_mask_incorrect,2186427.png
4,4511.png,297,6,452,240,face_with_mask,2974511.png
...,...,...,...,...,...,...,...
6950,01995_Mask_Mouth_Chin.jpg,0,128,0,128,face_with_mask_incorrect,01995_Mask_Mouth_Chin.jpg
6951,01996_Mask_Mouth_Chin.jpg,0,128,0,128,face_with_mask_incorrect,01996_Mask_Mouth_Chin.jpg
6952,01997_Mask_Chin.jpg,0,128,0,128,face_with_mask_incorrect,01997_Mask_Chin.jpg
6953,01998_Mask_Mouth_Chin.jpg,0,128,0,128,face_with_mask_incorrect,01998_Mask_Mouth_Chin.jpg


In [39]:
train.to_csv("train_inc.csv")
test.to_csv("test_inc.csv")

In [243]:
class_count = cropping.pivot_table(columns=['classname'], aggfunc='size')
print (class_count)

classname
face_no_mask                1115
face_other_covering          959
face_with_mask              2915
face_with_mask_incorrect    1100
dtype: int64
